In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt 
import io
import sys
import os
import tqdm

In [2]:
EPOCH = 10
BATCH_SIZE = 32
LR = 0.0001

In [3]:
T = transforms.ToTensor()
train_data = datasets.CIFAR10(root='~/newStuFile/ZHX/dataset/', train=True,transform=T,download=True)
test_data =datasets.CIFAR10(root='~/newStuFile/ZHX/dataset/',train=False,transform=T,download=True)
sys.path.append('..')
from torch.utils.data import DataLoader
train_loader = DataLoader(dataset=train_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=2)
test_loader = DataLoader(dataset=test_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = '2'

In [30]:
def train(model, train_loader, save_name) -> nn.Module:
    EPOCH = 10
    BATCH_SIZE = 32
    LR = 0.00001
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(),lr=LR)
    model = model.cuda()
    model.train()
    with tqdm(total = EPOCH*len(train_loader)) as pbar:
        for epoch in range(EPOCH):
            for i,data in enumerate(train_loader):
                inputs,labels = data
                inputs,labels = inputs.cuda(),labels.cuda()
                outputs = model(inputs)
                loss = criterion(outputs,labels)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                pbar.update(1)
                pbar.desc = '%f'%loss.item()
    torch.save(model,'../pts/%s'%save_name)
    return model

In [24]:
def test(model,test_loader) -> float:
    model.eval()
    correct,total = 0,0
    with tqdm(total = len(test_loader)) as pbar:
        for j,data in enumerate(test_loader):
            inputs,labels = data
            inputs,labels = inputs.cuda(),labels.cuda()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data,1)
            total =total+labels.size(0)
            correct = correct +(predicted == labels).sum().item()
            pbar.update(1)
    acc = 100.0*correct/total
    print('Accuracy：{:.4f}%'.format(acc))
    return acc

In [ ]:
from alexnet import alexnet_cifar
alexnet_model = alexnet_cifar()
alexnet_model = train(alexnet_model,train_loader,'cifar_alexnet.pt')
acc_alexnet = test(alexnet_model,test_loader)

In [ ]:
from vgg_19 import vgg19_cifar
vgg19_model = vgg19_cifar()
vgg19_model = train(vgg19_model,train_loader,'cifar_vgg19.pt')
acc_vgg19 = test(vgg19_model,test_loader)

In [15]:
from resnet_50 import resnet50_cifar
resnet50_model = resnet50_cifar()
resnet_50_model = train(resnet50_model,train_loader,'cifar_resnet_50')
acc_resnet50 = test(resnet_50_model,test_loader)

0.025620: 100%|██████████| 15630/15630 [1:35:02<00:00,  2.74it/s]


In [26]:
from mobilenet_v2 import mobilenetv2_cifar
mobilenetv2_model = mobilenetv2_cifar()
mobilenetv2_model = train(mobilenetv2_model,train_loader,'cifar_mobilenet_v2.pt')
acc_mobilenetv2 = test(mobilenetv2_model,test_loader)

100%|██████████| 313/313 [00:15<00:00, 20.36it/s]

Accuracy：79.2800%


In [27]:
from densenet_201 import densenet201_cifar
densenet201_model = densenet201_cifar()
densenet201_model = train(densenet201_model,train_loader,'cifar_densenet_201.pt')
acc_densenet201 = test(densenet201_model,test_loader)

100%|██████████| 313/313 [00:22<00:00, 13.71it/s]

Accuracy：64.7200%


In [31]:
ds_model = torch.load('../pts/cifar_densenet_201.pt')
ds_model = train(ds_model,train_loader,'cifar_densenet_201_09211002.pt')
acc_densenet201 = test(ds_model,test_loader)

100%|██████████| 313/313 [00:22<00:00, 13.81it/s]

Accuracy：67.4900%
